In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from stop_words import get_stop_words
from sklearn.model_selection import train_test_split, learning_curve, GridSearchCV, KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from yellowbrick.features.importances import FeatureImportances
import pickle
import os

### labelled large dataset (only subset)

In [21]:
df_l = pd.read_csv('../Data Files/scaledValidation - Sheet1.csv')
df_l.drop('Unnamed: 4', axis=1, inplace=True)
df_l = df_l[['VideoId', 'Manual Label']]
df_l['VideoId'] = df_l['VideoId'].str[32:]
print(df_l.shape)
df_l.head()

(84, 2)


,VideoId,Manual Label
0,cVhcbgxv3yE,f
1,iMtXqTmfta0,b
2,1dXWtgd5QYw,u
3,bSWEbaoxa7c,b
4,fch2yJ11pNg,b


### clean

In [22]:
df = pd.read_json('../../../data/uniqueScaledDataset.json')
print(df.shape)
df.head()

(38459, 14)


,viewCount,description,tags,predClass,channelId,videoId,comments,categoryId,commentCount,channelTitle,redirectedDomains,domains,title,completeDomains
0,298744,After months of searching for a constant onlin...,"[Money (TV Episode), How-to (Media Genre), Bus...",f,UCFqanUMpCMh54RmqRFh8hZw,dFotOUq0Z4E,"[over $34,000 was added to my paypal and that ...",22,240,ProfitClicksDelux,"[http://www.neobux.com/?r=dirven, https://imgu...","[neobux, imgur]",How to make money online just by clicking ads ...,"[http://www.neobux.com/?r=dirven, http://imgur..."
1,44244,hii.\nfriends.\n[[[[[[[[[[[[[[[[[[[[[[[[[[[[th...,"[earn money, how to, ads, paer, click, $5, per...",f,UC775w8nPdhDCVJ4IkCczVPw,xO5B6JcR5TQ,"[Earn Money\r\nhttps://youtu.be/NsqaXAMhl3k, n...",22,56,Trick Technology,[],[],earn money $2.to$5. ads click per day. in hindi.,[]
2,1066128,Create Free Bitcoin Wallet:- zebpay:- http://l...,[work from home],f,UCdWrX9xcYhCW4RwwYT22bpg,adh4xyzygBw,"[Sir ye site nhi chal rhi bhad ho gyi h kya, M...",26,2105,helpfromankit,"[http://link.zebpay.com/ref/REFANKI3843, http:...","[link.zebpay, myadvertisingpays]",view Only 10 Ads Per Day and Earn Money Every ...,"[http://link.zebpay.com/ref/REFANKI3843, http:..."
3,7567,My #1 Recommendation To Earning Income Online ...,"[make money online, tyler pratt, tylerpratt, e...",f,UCRfWuSls0R7jQazuddXiPvQ,H1KmFXp2Ymg,"[Didn't work, The real big money?? LOL cl...",26,9,Tyler Pratt,"[https://go.tylerpratt.com/, https://go.tylerp...","[go.tylerpratt, go.tylerpratt, facebook, insta...",Easy Side Online Job Clicking Ads and Making M...,"[http://go.tylerpratt.com/, http://go.tylerpra..."
4,9107,"Please watch: ""how to earn money from internet...","[Earn money By Click Ads, make money online, m...",f,UCupgYB_GTCWX6BLh-Ui_VFg,-YtKEcbVsKE,"[madarchod Frank hi, If you have website and e...",28,19,Tech 4 Digital,"[https://www.youtube.com/watch?v=DBZOdbHELXU, ...","[youtube, youtube, youtube, youtube, youtube, ...",Make money by Click ads क्लिक करके पैसे कमाओ ...,"[https://www.youtube.com/watch?v=DBZOdbHELXU, ..."


In [24]:
df_l = pd.merge(df, df_l, how='inner', left_on='videoId', right_on='VideoId')
print(df_l.shape)
df_l.head()

(80, 16)


,viewCount,description,tags,predClass,channelId,videoId,comments,categoryId,commentCount,channelTitle,redirectedDomains,domains,title,completeDomains,VideoId,Manual Label
0,50638,Earn $500 per sign up in the Los Angeles Area....,"[uber, lyft, rideshare, sign up, bonus, how to...",f,UCPmFGlgNyvdSwDIKVfv5qwg,z8-N86KHCEs,[Doesnt the link in the description make you m...,22,3,RideshareInvestor,[https://www.uber.com/a/join?exp_hvp=1&invite_...,[uber],Make money online with ppc ads. Sign up for Ub...,[https://partners.uber.com/i/8m87zcbpue],z8-N86KHCEs,f
1,38336,http://joinankur.com : join my team here to ge...,"[mass email magician, email list magician, col...",b,UCTiribC11Co_ft-r9wrunew,6tKK00krGoI,"[How does it work, Guys I swear I just got fre...",24,59,Ankur Agarwal,[http://joinankur.com/],[joinankur],"Collect Emails , Send Emails And Make Money On...",[http://joinankur.com],6tKK00krGoI,b
2,180,In it we learn how to implement Banner Ads in ...,"[how to add google adsense to android app, how...",f,UCV7hdDVonvC2ZOWNcHmM6eg,2ij6cSpEw2o,[Mobile number dena na muchhe kuchh puchh na hai],22,3,Easy Tut 4 U,[],[],Admob Banner ADS Tutorial Hindi by Easy Tut 4 U,[],2ij6cSpEw2o,f
3,14099,,"[Clicker Heroes, money glitch, clicker heroes ...",b,UC5QiV14A4UVAu8zwcO6L0VA,sSdiQ95l1rQ,"[I got 34,735#. Level 434, I when from 10k ...",20,9,NaCl Daddy,[],[],Clicker Heroes Easy MONEY GLITCH,[],sSdiQ95l1rQ,b
4,481,5 Easiest Ways To Make Money Online Internet I...,"[ways to make money online, india, hindi, onli...",f,UCniia-3AGf-HzVXooVcp1lw,AZyltjsXWBc,[],1,0,study tips,[https://www.youtube.com/watch?v=1yYpRSYkwIc&f...,"[youtube, youtube, youtube, youtube, youtube, ...",Top 5 Best Easy Ways To Earn Money Online Indi...,"[https://youtu.be/1yYpRSYkwIc, https://youtu.b...",AZyltjsXWBc,b


In [29]:
df_l['videoId'].is_unique

True

In [30]:
len(df_l['channelId'].unique())

79

In [31]:
df_l.dtypes

viewCount            object
description          object
tags                 object
predClass            object
channelId            object
videoId              object
comments             object
categoryId           object
commentCount         object
channelTitle         object
redirectedDomains    object
domains              object
title                object
completeDomains      object
VideoId              object
Manual Label         object
dtype: object

In [33]:
# df_l['classification'] = df_l['classification'].astype('category')
df_l['commentCount'][df_l['commentCount'] == 'No commentCount'] = np.NaN
df_l['commentCount'] = df_l['commentCount'].astype('float')
# df_l['category'] = df_l['category'].astype('category')
df_l['viewCount'][df_l['viewCount'] == 'No viewCount'] = np.NaN
df_l['viewCount'] = df_l['viewCount'].astype('float')
# df_l['linksCount'] = df_l['linksCount'].astype('float')
df_l['Manual Label'] = df_l['Manual Label'].astype('category')

C:\Users\ruote\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [36]:
df_l = df_l[['categoryId', 'channelId', 'channelTitle', 'Manual Label',
       'commentCount', 'comments', 'completeDomains',
       'description', 'domains',
       'redirectedDomains', 'tags', 'title',
       'videoId', 'viewCount']]

In [37]:
df.to_pickle('../Data Files/labelled_large.pkl')

### predictions

In [38]:
# Load grid search results
with open(f'results/RF_grid_5Kfold_results.pkl', 'rb') as f:
    cv_clf = pickle.load(f)